In [ ]:
from pyspark.sql import SQLContext
from dateutil.parser import parse
from datetime import datetime

def date_and_hour(s):
    dt = parse(s.replace('?',' '))
    hour = dt.hour
    return (dt.strftime("%Y-%m-%d"),hour)

sqlContext = SQLContext(sc)
df = sqlContext.read.format('com.databricks.spark.csv').options(header='true',inferschema='true').load('file:///home/oxclo/incidents/wind2014/*csv')
winddata = df.rdd
mapped = winddata.map(lambda x: (x.Station_ID, date_and_hour(x.Interval_End_Time),x.Wind_Velocity_Mtr_Sec, x.Ambient_Temperature_Deg_C))
filtered = mapped.filter(lambda (a,b,c,d): c != 0.0 and d != 0.0 and bool(c) and bool(d))
combinedstationdatetime = filtered.map(lambda (s,(d,h),v,t): ((s,d,h),(v,t)))
keyvalueone = combinedstationdatetime.map(lambda (station_date_hour,(v,t)): (station_date_hour,(v,t,1)))
sumdata = keyvalueone.reduceByKey(lambda (v1,t1,c1),(v2,t2,c2):(v1+v2,t1+t2,c1+c2))
averagedata = sumdata.map(lambda((s,d,h),(v,t,c)):((s,d,h),v/c,t/c))